In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup 
import re

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None)

In [2]:
cols = []
tables = pd.read_html("products_list.html")
df = tables[0]
cols = [item[-2] for item in df.columns]
df.columns = cols

def clear_string(string):
    pattern = r"\(.*\)"
    return re.sub(pattern, "", string)


products = [clear_string(product).replace("  ", " ").strip() for product in df["DESCRIPCION DEL PRODUCTO"].tolist() if type(product) == str and "total" not in product.lower() and "todos" not in product.lower()]



for product in products:
    print(product)

AL WATANIAH SABAH AL WARD EDP 100ML FEM
ARMAF CLUB DE NUIT INTENSE MAN EDT 105ML
LATTAFA AJWAD 100ML
LATTAFA AJWAD PINK TO PINK EDP 60ML
LATTAFA ASAD EDP 100ML
LATTAFA BADEE AL OUD AMETHYST 100ML
LATTAFA BADEE AL OUD FOR GLORY 100ML
LATTAFA FAKHAR EDP 100ML MASC
LATTAFA FAKHAR EXTRAIT GOLD EDP 100ML
LATTAFA FAKHAR ROSE GOLD EDP 100ML FEM
LATTAFA KHAMRAH BLACK EDP 100ML
LATTAFA KHAMRAH QAHWA EDP 100ML
LATTAFA MAYAR NATURAL INTENSE EDP 100ML
LATTAFA MUSAMAM OUD 100ML
LATTAFA PRIDE AJWAA EDP 90ML
LATTAFA PRIDE SHAHEEN SILVER EDP 100ML
LATTAFA YARA EDP 100ML
LATTAFA YARA CANDY EDP 100ML
LATTAFA YARA MOI EDP 100ML FEM
LATTAFA YARA TOUS EDP 100ML


# FIRST WEBPAGE 

https://www.comprasparaguai.com.br/

In [19]:
URL_TEMPLATE = "https://www.comprasparaguai.com.br/busca/?q={QUERY}"


products_data = [] 


for index, product in enumerate(products):
    response = requests.get(URL_TEMPLATE.format(QUERY=product.replace(" ", "+")))
    product_data = {}

    if index == 0:
        product_data["has_prime"] = True
    
    product_data["query"] = product
    product_data["status_code"] = response.status_code
    if response.status_code == 200:  
        soup = BeautifulSoup(response.text, "lxml")
        items = soup.select("div.promocao-item-preco-oferta.flex.column a")
        price_box = soup.select_one(".preco-dolar strong")
        if price_box:
            product_data["price"] = price_box.text.strip()
        else:
            price_box = soup.select_one(".price-model strong")
            if price_box:
                product_data["price"] = price_box.text.strip()
            else:
                product_data["price"] = ""
        product_data["items"] = items
        product_data["items_count"] = len(items)
        for item in items:
            if item.select_one("button"):
                href = item.get("href")  
                if href:  
                    product_data["link"] = href
                    break
                else:
                    product_data["link"] = None
            else:
                href = item.get("href")  
                if href:  
                    product_data["link"] = href
                product_data["link"] = href
                print(f"Erro ao buscar o produto '{product}': {response.status_code}")
                
        products_data.append(product_data)
    else:
        print(f"Erro ao buscar o produto '{product}': {response.status_code}")


Erro ao buscar o produto 'LATTAFA BADEE AL OUD FOR GLORY 100ML': 200


In [20]:
BASE_URL = "https://www.comprasparaguai.com.br"

for index, data in enumerate(products_data):
    print("="*100)
    try:
        if data["link"]:
            print(BASE_URL + data["link"])
            response = requests.get(BASE_URL + data["link"])
            soup = BeautifulSoup(response.text, "lxml")
            listed_items = soup.select(".promocao-produtos-item")
            for item in listed_items:
                store = item.select_one("img.lozad.store-image").get("alt").lower().strip().replace(" ", "_")
                price = item.select_one(".promocao-item-preco-oferta.flex.column strong").text.strip()
                products_data[index][f"{store}_name"] = store
                products_data[index][f"{store}_price"] = price
                store_link = item.select_one(".btn.btn-blue.btn-store-redirect").get("href")
                print(data["query"])
                print(store, price)
                print(store_link)
                # if store_link:
                #     products_data[index][f"{store}_link"] = store_link
                # else:
                #     products_data[index][f"{store}_link"] = None
                
    except Exception as e:
        continue
    
    print("="*100)

https://www.comprasparaguai.com.br/perfume-al-wataniah-sabah-al-ward-eau-de-parfum-feminino-100ml_44197/
AL WATANIAH SABAH AL WARD EDP 100ML FEM
elegancia_company US$ 24,99
https://www.eleganciacompany.com/#/productos/perfumes/2057
AL WATANIAH SABAH AL WARD EDP 100ML FEM
shopping_china US$ 28,00
https://www.shoppingchina.com.py/producto/835754
AL WATANIAH SABAH AL WARD EDP 100ML FEM
toku_importados US$ 26,00

AL WATANIAH SABAH AL WARD EDP 100ML FEM
multipass US$ 26,00
https://multipass.com.py/destaque/5938-perfume-al-wataniah-sabah-al-ward-edp-femenino-100ml.html
AL WATANIAH SABAH AL WARD EDP 100ML FEM
nissei US$ 28,00
https://nissei.com/br/catalog/product/view/id/138895
AL WATANIAH SABAH AL WARD EDP 100ML FEM
cellshop US$ 27,00
https://cellshop.com/p-al-wataniah-f-sabah-al-ward-100ml-edp
AL WATANIAH SABAH AL WARD EDP 100ML FEM
mega_eletrônicos US$ 24,00
https://www.megaeletronicos.com/br/p/1431629/perfume-feminino-al-wataniah-sabah-al-ward-edp-100-ml
AL WATANIAH SABAH AL WARD EDP 100M

In [21]:
import pandas as pd

df = pd.DataFrame(products_data)
df.drop(columns=["items", "items_count", "link", "status_code", "has_prime"], inplace=True)

In [22]:
df

,query,price,elegancia_company_name,elegancia_company_price,shopping_china_name,shopping_china_price,toku_importados_name,toku_importados_price,multipass_name,multipass_price,nissei_name,nissei_price,cellshop_name,cellshop_price,mega_eletrônicos_name,mega_eletrônicos_price,pioneer_international_name,pioneer_international_price,mobile_zone_name,mobile_zone_price,la_petisquera_name,la_petisquera_price,madrid_center_name,madrid_center_price,star_company_name,star_company_price,pontocom_name,pontocom_price,atlantico_shop_name,atlantico_shop_price,victoria_store_name,victoria_store_price,macedonia_name,macedonia_price,alborada_name,alborada_price,atacado_connect_name,atacado_connect_price,genove_name,genove_price,epic_importados_name,epic_importados_price,casa_americana_name,casa_americana_price,mega_eletro_name,mega_eletro_price,smart_store_name,smart_store_price,world_of_vape_name,world_of_vape_price,asm_group_name,asm_group_price,dr_cell_atacado_name,dr_cell_atacado_price,charme_perfumaria_name,charme_perfumaria_price,prime_shop_name,prime_shop_price,roma_shopping_name,roma_shopping_price,agatres_name,agatres_price,aroma_store_s.a_name,aroma_store_s.a_price,super_games_name,super_games_price,terra_nova_name,terra_nova_price,adeshop_name,adeshop_price,brands_for_less_name,brands_for_less_price,sampa_vapor_name,sampa_vapor_price,top_house_name,top_house_price,computech_name,computech_price,mini_carlos_name,mini_carlos_price,mario_cell_name,mario_cell_price,eshop_name,eshop_price,new_zone_name,new_zone_price,matrix_importados_name,matrix_importados_price,centro_de_motorola_name,centro_de_motorola_price
0,AL WATANIAH SABAH AL WARD EDP 100ML FEM,,elegancia_company,"US$ 24,99",shopping_china,"US$ 28,00",toku_importados,"US$ 26,00",multipass,"US$ 26,00",nissei,"US$ 28,00",cellshop,"US$ 27,00",mega_eletrônicos,"US$ 24,00",pioneer_international,"US$ 24,00",mobile_zone,"US$ 26,00",la_petisquera,"US$ 26,00",madrid_center,"US$ 26,00",star_company,"US$ 23,00",pontocom,"US$ 28,00",atlantico_shop,"US$ 26,00",victoria_store,"US$ 32,00",macedonia,"US$ 23,50",alborada,"US$ 27,00",atacado_connect,"US$ 23,00",genove,"US$ 24,00",epic_importados,"US$ 25,00",casa_americana,"US$ 30,00",mega_eletro,"US$ 25,50",smart_store,"US$ 33,00",world_of_vape,"US$ 25,00",asm_group,"US$ 25,00",dr_cell_atacado,"US$ 25,00",charme_perfumaria,"US$ 28,00",prime_shop,"US$ 23,50",roma_shopping,"US$ 29,00",agatres,"US$ 26,50",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARMAF CLUB DE NUIT INTENSE MAN EDT 105ML,,elegancia_company,"US$ 35,00",shopping_china,"US$ 32,00",toku_importados,"US$ 27,00",multipass,"US$ 27,90",nissei,"US$ 40,00",cellshop,"US$ 41,00",mega_eletrônicos,"US$ 27,50",pioneer_international,"US$ 24,00",mobile_zone,"US$ 28,00",NaN,NaN,madrid_center,"US$ 28,00",star_company,"US$ 23,50",pontocom,"US$ 30,00",NaN,NaN,victoria_store,"US$ 30,00",macedonia,"US$ 24,25",NaN,NaN,atacado_connect,"US$ 23,50",genove,"US$ 24,50",epic_importados,"US$ 25,00",NaN,NaN,mega_eletro,"US$ 27,50",NaN,NaN,NaN,NaN,asm_group,"US$ 30,00",dr_cell_atacado,"US$ 25,00",NaN,NaN,prime_shop,"US$ 23,50",roma_shopping,"US$ 30,00",agatres,"US$ 27,50",aroma_store_s.a,"US$ 26,00",super_games,"US$ 23,50",terra_nova,"US$ 43,00",adeshop,"US$ 27,00",brands_for_less,"US$ 28,00",sampa_vapor,"US$ 35,00",top_house,"US$ 25,50",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LATTAFA AJWAD 100ML,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LATTAFA AJWAD PINK TO PINK EDP 60ML,,elegancia_company,"US$ 25,00",shopping_china,"US$ 37,50",NaN,NaN,NaN,NaN,nissei,"US$ 25,00",NaN,NaN,mega_eletrônicos,"US$ 19,00",NaN,NaN,mobile_zone,"US$ 

In [23]:
df.to_csv("products_data.csv", index=False)

In [ ]:
BASE_URL = "https://www.comprasparaguai.com.br"

for data in products_data:
    try:
        if data["link"]:
            print(BASE_URL + data["link"])
            response = requests.get(BASE_URL + data["link"])
            soup = BeautifulSoup(response.text, "lxml")
            listed_items = soup.select(".promocao-produtos-item")
            for item in listed_items:
                store = item.select_one("img.lozad.store-image").get("alt")
                price = item.select_one(".promocao-item-preco-oferta.flex.column strong").text.strip()
                print(store, price)
                # if item.select_one("img[alt='Prime shop']"):
                #     print("Achou prime")
                #     data["has_prime"] = True
                #     data["prime_price"] = item.select_one(".promocao-item-preco-oferta.flex.column strong").text.strip()
                # if item.select_one("img[alt='Macedonia']"):
                #     print("Achou macedonia")
                #     data["has_macedonia"] = True
                #     data["macedonia_price"] = item.select_one(".promocao-item-preco-oferta.flex.column strong").text.strip()
                # if item.select_one("img[alt='Atacado Connect']"):
                #     print("Achou atacado connect")
                #     data["has_atacado_connect"] = True
                #     data["atacado_connect_price"] = item.select_one(".promocao-item-preco-oferta.flex.column strong").text.strip()
    except Exception as e:
        continue
    
    print("="*100)

In [ ]:
for data in products_data:
    for key, value in data.items():
        print(f"{key}: {value}")
    print("="*100)

In [ ]:
empty_blocks